In [1]:
import numpy  as np
import pandas as pd
from metrics import MSE
from tqdm import tqdm

In [2]:
X_train = pd.read_pickle("../../../../Datasets/temporal/final/X_train.pkl").drop(columns=["gPlusUserId", "gPlusPlaceId"])
c = X_train.columns.tolist()
X_val = pd.read_pickle("../../../../Datasets/temporal/final/X_val.pkl").drop(columns=["gPlusUserId", "gPlusPlaceId"])
X_test = pd.read_pickle("../../../../Datasets/temporal/final/X_test.pkl").drop(columns=["gPlusUserId", "gPlusPlaceId"])
y_train = pd.read_pickle("../../../../Datasets/temporal/final/y_train.pkl").to_numpy()
y_val = pd.read_pickle("../../../../Datasets/temporal/final/y_val.pkl").to_numpy()
y_test = pd.read_pickle("../../../../Datasets/temporal/final/y_test.pkl").to_numpy()

In [3]:
X_train.insert(X_train.shape[1], "rating", y_train.T[0])

In [4]:
state_columns = [i for i in c if "state" in i]

In [5]:
mean_per_state_dict = {}
for state in state_columns:
    state_df = X_train.groupby(state).agg({"rating": "mean"}).reset_index()
    mean_per_state_dict[state] = state_df[state_df[state] == 1]["rating"].item()

In [6]:
mean_per_state_dict

{'state_AL': 3.86983795086252,
 'state_AR': 3.8375738300772375,
 'state_AZ': 3.863553422991147,
 'state_CA': 3.9369029837785265,
 'state_CO': 3.941110350894341,
 'state_CT': 3.8907804969643864,
 'state_DE': 3.941062176165803,
 'state_GA': 3.8676523404764755,
 'state_IA': 3.8667042677194963,
 'state_ID': 3.955899419729207,
 'state_IL': 3.901098901098901,
 'state_IN': 3.9859421186498936,
 'state_KS': 3.8559508124725514,
 'state_KY': 3.8765534382767193,
 'state_LA': 3.7815126050420167,
 'state_MA': 3.9952135493372607,
 'state_MD': 3.8432499798986894,
 'state_ME': 3.9630931458699474,
 'state_MI': 3.8935883193143948,
 'state_MN': 3.8672050705377226,
 'state_MO': 3.9023027927486527,
 'state_MS': 3.667953667953668,
 'state_MT': 3.971357409713574,
 'state_NC': 3.910533677641489,
 'state_ND': 3.5844155844155843,
 'state_NE': 3.5725388601036268,
 'state_NH': 4.092816787732042,
 'state_NJ': 3.936513551094089,
 'state_NM': 3.861819235225956,
 'state_NV': 3.869701726844584,
 'state_NY': 3.830705796

In [7]:
y_pred_val = []
for i in tqdm(range(len(X_val))):
    for j in range(len(X_val.iloc[i][state_columns])):
        if X_val.iloc[i][state_columns][j] == 1:
            y_pred_val.append(mean_per_state_dict[state_columns[j]])
            break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161526/161526 [11:17<00:00, 238.38it/s]


In [8]:
MSE(predictions=y_pred_val, labels=y_val)[0]

1.721785866921825

In [9]:
y_pred = []
for i in tqdm(range(len(X_test))):
    for j in range(len(X_test.iloc[i][state_columns])):
        if X_test.iloc[i][state_columns][j] == 1:
            y_pred.append(mean_per_state_dict[state_columns[j]])
            break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161526/161526 [10:49<00:00, 248.53it/s]


In [10]:
MSE(predictions=y_pred, labels=y_test)[0]

2.529536257440224